2018/11/24：　https://www.quantopian.com/lectures/factor-analysis-with-alphalens
からクローンしてきた

# φ(..)メモメモ

+ Alphalensでは良いFactorを見つけるのが目的
+ 良いFactorとは、あるファクターがとある動きをした場合、とある銘柄群がオーバーパフォームし、とある銘柄群がアンダーパフォームする事を示すファクター
+ つまりFactorとは、銘柄をランキングする指標
+ Alphalensではマーケットインパクトやトランザクションコストは考慮されていない

    

# Factor Analysis

by Maxwell Margenot, Gil Wassermann, James Christopher Hall, and Delaney Granizo-Mackenzie.

Part of the Quantopian Lecture Series:

* [www.quantopian.com/lectures](https://www.quantopian.com/lectures)
* [https://github.com/quantopian/research_public](https://github.com/quantopian/research_public)

---

How can we tell whether an alpha factor is good or not? Unfortunately, there is no cut-off that tells you whether a factor is objectively useful. Instead, <font color=red>we need to compare a particular factor to other options before deciding whether to use it</font>. Our end goal in defining and selecting the best factors is to use them to rank stocks in a long-short equity strategy, covered elsewhere in the [lecture series](https://www.quantopian.com/lectures#Long-Short-Equity). The more independent predictive the factors we use, the better our **ranking scheme** and our overall strategy will be.

alpha factor が良い指標と言えるかどうか、どうやって決めればいいでしょう？
客観的にあるファクターが便利だ、とさくっと言える方法はありません。
他のファクターと比べて決める必要があります。
私達のゴールは、ロングショート戦略で使う銘柄を、ファクターでランキングして決める事です。（くわしくは [lecture series](https://www.quantopian.com/lectures#Long-Short-Equity)）
より独立したファクターを使えば、より良いランキングが得られ、結果いいストラテジーが得られるでしょう。


What we want when comparing factors is to make sure the chosen signal is actually predictive of <font color=red>**relative price movements**</font>. We do not want to predict the absolute amount the assets in our universe will move up or down. <font color=red>We only care that we can choose assets to long that will do better than the assets we short.</font> In a long-short equity strategy, we hold a long basket and a short basket of assets, determined by the factor values associated with each asset in our universe. If our ranking scheme is predictive, this means that assets in the top basket will tend to outperform assets in the bottom basket. As long this spread is consistent over time our strategy will have a positive return.

複数のファクターを比べる時必要な事は、選んだシグナルが価格変化を比較的よく予測していると判断できることです。
それは、選んだユニバースの実際の価格推移ではなく、<font color=red>買いのバスケットに入れた銘柄が売りのバスケットに入れた銘柄よりもより良い動きをすることです。</font>ロングショートストラテジーでは、ユニバースの中の各銘柄がから得られるファクターの値で、ロングもしくはショートのバスケットを作ります。ランキングがうまく出来ているならば。ファクターのトップ銘柄が、ボトム銘柄をアウトパフォームするバスです。このスプレッドがコンスタントに続けば、私達のストラテジーはポジティブリターンをもたらします。


An individual factor can have a lot of moving parts to assess, but ideally it should be independent of other factors that you are already trading on in order to keep your portfolio diverse. We discuss the reasoning for this in [this lecture](https://www.quantopian.com/lectures#Position-Concentration-Risk). 


（複数のファクターを持っているとして）各ファクターはそれぞれの動きをする。理想的には各ファクターはそれぞれに独立している方がいい。そうすれば、ポートフォリオに多様性が保たれるから。（多様性は、Riskを抑えるというQuantopianの考え方）

In this lecture, we detail and explain relevant statistics to evaluate your alpha factor before attempting to implement it in an algorithm. What's important to keep in mind is that all the metrics provided here are relative to other factors you may be trading or evaluating. 

このレクチャで、あなたのアルファファクタを評価するための統計を説明しています。
それはアルゴリズムに書き下ろす前にするべきことです。
重要なことは、ここにあるすべての評価基準（metrics) は、すべて他のファクターに関係するもので、それもまたあなたが取引したり評価したりするものかもしれません。

Let's have a look at a factor and try to assess its viability. We will calculate the factor values using Pipeline, so make sure you check out the [tutorial](https://www.quantopian.com/tutorials/pipeline) if you are unfamiliar with how Pipeline works.

まずは手始めに、ある一つのファクターを可能性（能力、潜在的な価値）を評価してみましょう。
計算にはパイプラインを使います。
パイプラインをよく知らない人はこちらを見てください。
[tutorial](https://www.quantopian.com/tutorials/pipeline) 


In [ ]:
import numpy as np
import pandas as pd
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns
from quantopian.pipeline.classifiers.fundamentals import Sector  
from quantopian.pipeline.filters import QTradableStocksUS
from time import time

### Momentum

Here we will be using a **momentum** factor as our example. Momentum factors are a very common form of alpha factor and they come in many shapes and sizes. They all try to get at the same idea, however, that securities in motion will stay in motion. Momentum factors try to quantify trends in financial markets and to "ride the wave", so to speak.

ではモメンタムを例にしてみましょう。
モメンタムファクターは、とても良く使われるアルファファクターの形（form)で、いろんな形や大きさになりえます。
モメンタムファクターはマーケットの”定量的な”トレンドをつかむことから、「波にのる」事に使われます。

Let's say that we suspect that a momentum factor could potentially be predictive of stock returns. We define it as a `CustomFactor` so that we can easily pull its values when we run our Pipeline. We should get a factor value for every security in our universe.
例えば、株の収益をモメンタムが潜在的に予測する、と仮定したとしましょう。
それを、 `CustomFactor` と定義して、パイプラインに流すことができます。
そうすれば、ユニバースに入っている全銘柄に対して、 `CustomFactor` として定義したモメンタムの数値がとれます。

In [ ]:
class MyFactor(CustomFactor):
        """ Momentum factor """
        inputs = [USEquityPricing.close,
                  Returns(window_length=126)]
        window_length = 252
        
        ## （現在から21日前ー252日前）/252日前　から（現在から1日前ー21日前）/21日前　で割ったものを標準化
        ## つまり、一ヶ月前と今を比較して、一ヶ月前と一年前を比較して、この2つ数を比較した

        def compute(self, today, assets, out, prices, returns):
            out[:] = ((prices[-21] - prices[-252])/prices[-252] -
                      (prices[-1] - prices[-21])/prices[-21]) / np.nanstd(returns, axis=0)

This momentum factor takes the change in price over the past year, up until a month ago, and standardizes it over the the change in price over the last month. This allows it to account for any changes over the past month and use them to temper its expectations of future movement.

このモメンタムは一ヶ月前と今を比較して、一ヶ月前と一年前を比較して、この2つ数を比較して、標準化したもの。


## Judging a Factor with Alphalens

In order to judge whether a factor is viable, we have created a package called Alphalens. Its source code is available on [github](http://github.com/quantopian/alphalens) if you want to get into the nitty-gritty of how it works. We use Alphalens to create a "tear sheet" of a factor, similar to how we use [pyfolio](http://github.com/quantopian/pyfolio) to create a tear sheet for analyzing backtests.

## アルファレンズ使うで

とあるファクターが使えるやつかどうか判断するためにアルファレンズ作りました。
コードはここにあるよ。[github](http://github.com/quantopian/alphalens) 
nitty-grittyに見たい人はコード読んで。（nitty-gritty核心に迫る）
アルファレンズは、「tear sheet」と呼ばれるものを出すためにつくった。[pyfolio](http://github.com/quantopian/pyfolio)の tear sheet と似てるやつ。



In [ ]:
import alphalens as al

Alphalens takes your factor and examines how useful it is for predicting relative value through a collection of different metrics. It breaks all the stocks in your chosen universe into different quantiles based on their ranking according to your factor and analyzes the returns, information coefficient (IC), the turnover of each quantile, and provides a breakdown of returns and IC by sector.

Alphalensは、あなたのファクターがリラティブバリューをどのように予測するか、テストするのに便利です。そのためいいろんな判断指標をつかいます。ユニバースに入っている銘柄を分解しランキングごとに分位して、収益、IC、それぞれの分位でのTurnover、セクターごとの収益とICを分析します。

Throughout the course of this lecture we will detail how to interpret the various individual plots generated by an `Alphalens` tear sheet and include the proper call to generate the whole tear sheet at once at the end.

このノートで`Alphalens`が作ったtear sheet の結果の説明とTear Sheetの作り方（一部作成と、全部一度に作成する方法がある）を書きます


([レラティブ・バリューとは｜金融経済用語集](https://www.ifinance.ne.jp/glossary/investment/inv277.html)
レラティブ・バリューは、「レラティブバリュー戦略（相対価値投資戦略）」とも呼ばれ、主にヘッジファンドにおいて、株式や債券、通貨、コモディティなどのロング（買い）とショート（売り）を組み合わせことによって生ずる価格差で収益を上げる投資戦略をいいます。)

（I C（Information Coefficient）は「実現した」アクティブ・リターンと「事前に予測していた」アクティブ・リターン（アルファ）の相関）

## Sector Codes

These are the possible sector codes for each security, as given by Morningstar. We will use this dictionary to help categorize our results as we walk through a factor analysis so that we can break out our information by sector.

## 業種番号

Morningstarはセクターをセクターコードで出すので、それを実際の業種名に変更するための辞書

In [ ]:
MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}

## Defining a universe

As always, we need to define our universe. In this case we use the QTradableStocksUS, as seen in the forums [here](https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus).

## ユニバースを設定

QTradableStocksUSを使う

In [ ]:
universe = QTradableStocksUS()

## Getting Data

Now we will pull values for our factor for all stocks in our universe by using Pipeline. We also want to make sure that we have the sector code for each individual equity, so we add `Sector` as another factor for our Pipeline. Note that running the Pipeline may take a while.

パイプラインを使って、私達のユニバースの銘柄を引き出しましょう。
`Sector`も引き出しておけば、あとで上記の辞書を使って変換できます。
パイプラインの実行にはちょっと時間かかるで。


In [ ]:
pipe = Pipeline(
    columns = {
            'MyFactor' : MyFactor(mask=universe),
            'Sector' : Sector()
    },
    screen=universe
)

start_timer = time()
results = run_pipeline(pipe, '2015-01-01', '2016-01-01')
end_timer = time()
results.fillna(value=0);

In [ ]:
print "Time to run pipeline %.2f secs" % (end_timer - start_timer)

In [ ]:
results.head()

Let's take a look at the data to get a quick sense of what we have.

In [ ]:
my_factor = results['MyFactor']
print my_factor.head()

Our `my_factor` variable contains a pandas `Series` with a factor value for each equity in our universe for each point in time.

Here we create another `Series` that contains sector codes for each equity instead of factor values. This is categorical data that we will use as a parameter for `Alphalens` later.

`my_factor`、`sectors` とシリーズとして取り出しといて、`Alphalens` に渡す準備

In [ ]:
sectors = results['Sector']

While our universe is defined to consist of 1500 stocks, the actual number of unique stocks that we end up ranking will likely  be greater than this due to stocks passing in and out of our filters. For this reason, we grab pricing data for any stock that may have been in our Pipeline at some point to make sure that we have all of the data that we might need.

`QTradableStocksUS()` には1500の銘柄が入ってますが、数えるとそれ以上です。というのも毎日入れ替えを行っているから。なので、取得した期間に入っていた銘柄すべてに対して、株価を取得するために以下の処理をします。
ここで取得したデータは、あとでAlphalensに渡します。

In [ ]:
asset_list = results.index.levels[1]
len(asset_list)

In [ ]:
prices = get_pricing(asset_list, start_date='2015-01-01', end_date='2016-02-01', fields='open_price')

In [ ]:
prices.head()

（一部NaNが見えるのは、その銘柄は指定期間初日頃にはまだ `QTradableStocksUS` には入っていなかったということ）

# Alphalens Components

Now that we have the basic components of what we need to analyze our factor, we can start to deal with `Alphalens`. Note that we will be breaking out individual components of the package, so this is not the typical workflow for using an `Alphalens` tear sheet.

# アルファレンズコンポネント

さて、私達のファクターを分析する基本的な部品を見てみましょう。アルファレンズの出番です。
（今から、一つずつコンポネントを見ていきますが、いつもはこんな事せずいちどにやっちゃっていいですからね）


First we get our factor categorized by sector code and calculate our forward returns. The forward returns are the returns that we would have received for holding each security over the day periods ending on the given date, passed in through the `periods` parameter. In our case, and by default, we look $1$, $5$, and $10$ days in advance. We can consider this a budget backtest. The tear sheet does not factor in any commission or slippage cost, rather, it only considers values as if we had magically already held the specified equities for the specified number of days up to the current day.

まず、セクターコードてカテゴライズしてフォワード収益を計算。
フォワード収益とは、 `periods` パラメータで渡した日数の収益。
デフォルトでは、1，5，10日。
これをbudget backtestと考えていいでしょう。（？）
tear sheet はコストやスリッページを因数に分解しないので、取得出来たものとして計算



In [ ]:
periods = (1, 5, 10)

In [ ]:
factor_data = al.utils.get_clean_factor_and_forward_returns(factor=my_factor,
                                                            prices=prices,
                                                            groupby=sectors,
                                                            groupby_labels=MORNINGSTAR_SECTOR_CODES,
                                                            periods=periods)

The `factor` variable here is similar to the `my_factor` variable above. It has a factor value for every equity in our universe at each point in time. Our `Alphalens` function here has also provided a sector grouping to go along with the factor value.

ここで出来た `factor_data `を tear sheet作成関数に渡して、分析する

In [ ]:
factor_data.head()

As explained above, the forward returns are the returns that we would have received for holding each security for the specified number of days, ending on the given date. These, too, are broken out by sector.

forward returns は、その日からｎ日後の収益率。セクターごとにも出せます。

This function also separates our factor into quantiles for each date, replacing the factor value with its appropriate quantile on a given day. Since we will be holding baskets of the top and bottom quantiles, we only care about the factor insofar as it relates to movement into and out of these baskets.


トップ分位とボトム分位を保有するので、バスケットに出入りがあるファクターだけ気にかけておけばいい。

Alphalens provides three categories of analysis on alpha factors:

* Returns
* Information
* Turnover

Each of these topics is covered in its own separate tear sheet as well as an all-encompassing full tear sheet.

アルファレンズは、アルファファクターの分析を3つのカテゴリーに分けてやってくれる

* 収益
* 情報
* Turnover

それぞれの、トピックはそれぞれの tear sheet に分けてある。またfull tear sheet であれば全部入る。


### Returns Tear Sheet

If we are solely interested in returns, we can create a tear sheet that only contains the returns analysis. The following code block generates all of our returns plots once we have stored the forward returns data:

### 収益 Tear sheet 

リターンにだけ興味あるなら、これだけ分析すればいい。続くコードは、フォワードリターンの結果をプロットしてくれる

In [ ]:
al.tears.create_returns_tear_sheet(factor_data, by_group=True);

### Returns Tear Sheet Breakdown
### 収益 Tear Sheet 解説

#### $\alpha$ and $\beta$ Point Estimates Chart

Now we calculate the $\alpha$ and $\beta$ of our factor with respect to the market. These are calculated by creating a regression of the market returns for each period against a long-short factor portfolio and extracting the parameters. These signify the excess return associated with our factor and the market beta, respectively.

最初に出てくるテーブル。

$\alpha$ と $\beta$ は、マーケットに対するアルファとベータ。いつものやつ。



#### Returns Point Estimates Chart

These point estimates were also calculated agnostic of the sector groupings so they give us an overall view of what our spread would look like if we traded this factor with a long-short equity algorithm and didn't examine which sectors those returns were coming from.


<img src="https://3.bp.blogspot.com/-pq-LLjyPoqo/WwJZnND_U-I/AAAAAAABMGc/RPqrox3dUdwSqUo6FWF4zhxzor8kYMIvACLcBGAs/s800/character_turtle_oyako_mago.png" alt="" width="100" height="100">これもテーブルの中のほか3つ






#### Mean Period-Wise Return by Quantile

The quantized factor is used to create separate baskets, with returns for each basket calculated over each given day period. We get this information both as bar charts and as violin plots.

The bar charts provide us the mean return for each factor quantile, for each time horizon. This gives us a set of point estimates to get a preliminary understanding of how the factor value tracks with returns.

The primary difference between a bar chart and a violin plot is that a violin plot shows the density of our data. The fatter the violin, the higher the density of returns in that region. Here we plot the violins for the $1$, $5$, and $10$ day forward returns for each quantile.

#### Mean period wise Return by Factor Quantile 
最初の棒グラフとバイオリングラフ。

与えられた日数別に収益の「平均値」を分位に分けて表示。とりあえずどんな収益が得られるか理解する（preliminary understanding）のに使う。

棒とバイオリンの違いは、バイオリンはどこにその収益が溜まっているかを確認できる。平均値では見えないところ。





#### Factor-Weighted Long/Short Portfolio Cumulative Return

These plots show the cumulative returns of a factor-weighted cross-sectional equity strategy based on the input factor. This holds the top quantile long and the bottom quantile short and simulates how that portfolio would perform without slippage or commissions. This shows the performance of the factor as a whole, which is always important to consider.

#### Factor-Weighted Long/Short Portfolio Cumulative Return

バイオリンの次にある折れ線グラフ

上のMean Periodでいう1をショートして5をロングする戦略でいって、1日の収益を積み上げて言った結果を折れ線グラフにしてる。（スリッページと手数料は考慮なし）



#### Cumulative Return by Quantile

factor_data = al.utils.get_clean_factor_and_forward_returns(factor=my_factor,
                                                            prices=prices,
                                                            groupby=sectors,
                                                            groupby_labels=MORNINGSTAR_SECTOR_CODES,
                                                            periods=periods)This next plot aggregates the returns of each individual quantile into a plot of cumulative returns separated by basket for the 1-period forward returns. What we want here is to see five discrete "fingers" with few to no crossovers. This will give us an idea of which quantiles tend to drive the returns (ideally the first and fifth).

その次の折れ線グラフ、1から5の線が入ってるグラフ

1日の収益を折れ線グラフにしたもの。トップとボトムだけじゃなく、全クオンタイルをグラフにしている。理想的には５と１をロングショートするのがいいけど、実際はどのクオンタイルが収益を上げているのか見るのにべんり

このあとは、
５日、１０日、と指定した日数の同じ結果が表示されている

##### Top Minus Bottom Quantile Mean Return 
TopからBottomをひいた結果を（積算せずに）表示、その移動平均を表示。



#### Quantile Mean Return in BPS

Next we calculate the basis points of the spread, based on subtracting the mean return of the lowest quantile from the mean return of the highest quantile (simulating going short on the lowest and long on the highest). **We also get the error and plot it all together, giving us a time series of the basis points with confidence intervals for each time period.**

最初のテーブルに戻って、Mean Period Wise Return Top Quantile (bps) 	以下３つ。

今まで見た結果をBasis Pointでの収益表示。（トップ平均からボトム平均をひいた結果）



<img src="https://3.bp.blogspot.com/-pq-LLjyPoqo/WwJZnND_U-I/AAAAAAABMGc/RPqrox3dUdwSqUo6FWF4zhxzor8kYMIvACLcBGAs/s800/character_turtle_oyako_mago.png" alt="" width="100" height="100">


#### Sector Breakdown of Returns

`Alphalens` provides a breakdown of returns by group, provided that you feed it a set of categorical values (such as industry labels), using the `by_group` flag. While it is good to consider breakdowns by quantile, it is also important to see how your factor is exposed to the different facets of the market. This is a good way to assess if your factor is behaving as it should in your universe. For example, if you intend your universe to only touch a certain sector, it is worthwhile to confirm that your factor and universe indeed only touch that sector.

#### Sector Breakdown of Returns

```python

factor_data = al.utils.get_clean_factor_and_forward_returns(
    factor=my_factor,
    prices=prices,
    groupby=sectors,
    groupby_labels=MORNINGSTAR_SECTOR_CODES,
    periods=periods)

```

`Alphalens` ではgroupbyで渡したグループに分けて結果を表示できる。



## Information Coefficient

We use the information coefficient (IC) to assess the predictive power of a factor. The IC of a factor is its Spearman Rank Correlation. For more background on the mathematics associated with the IC, check out the [Spearman Rank Correlation Lecture](https://www.quantopian.com/lectures#Spearman-Rank-Correlation). To break it down, we calculate the IC between the factor values and the forward returns for each period. The IC assesses the monotonic relationship between factors and returns. What this means, intuitively, is that it provides a measure for whether higher factor values can be associated with higher returns.

A higher IC indicates that higher factor values are more closely associated with higher return values (and lower factor values with lower return values). A very negative IC indicates that higher factor values are closely associated with lower return values. An IC of $0$ indicates no relationship.


## Information Coefficient
ファクターがどれほど予測能力があるかをICを使って評価。
Spearman Rank Correlationを使って評価している。Spearmanに関しては、[Spearman Rank Correlation Lecture](https://www.quantopian.com/lectures#Spearman-Rank-Correlation)。
それぞれの期間において、ファクターと収益の関係を計算している。
ICは、”monotonic relationship”　を評価している。monotonic relationshipとは、一つの値が上がれば、もう一つも上がるか、また、一つの値が上がれば、もう一つは下がるか、という関係性のこと。





### Information Tear Sheet

Similarly to the returns tear sheet, we can also create a tear sheet that just gives us data on the information coefficient.

収益tear sheet と同様、ICの tear sheet を作成

In [ ]:
al.tears.create_information_tear_sheet(factor_data, 
                                       by_group=True)

#### IC Point Estimates

This first table provides us with statistics around the distribution of the information coefficient for each time horizon. This includes mean, standard deviation, skewness, and kurtosis, to give us a clear view of how the distribution is shaped.

これは、各期間ごとのIC。平均、偏差、スキュー、尖度


尖度（せんど、英: kurtosis)は、確率変数の確率密度関数や頻度分布の鋭さを表す数字である。 正規分布と比べて、尖度が大きければ鋭いピークと長く太い裾を持った分布を持ち、尖度が小さければより丸みがかったピークと短く細い尾を持った分布であるという事が判断


#### IC Time Series

Here we plot the IC as a time series for each period along with a 1-month moving average to smooth it out. What we want here is consistency over time and a consistently informative signal.

ICの時系列折れ線グラフ。



#### IC Histograms

Histograms are good to show us the distribution of the IC. These will clearly show any strange outliers and how they affect the overall curve.

ヒストグラムでICの分散を見ます。外れ地がどのくら影響してるか見ます。



#### IC QQ-Plots

A QQ-plot compares the distribution of the IC to the normal distribution. It plots the quantiles of one distribution against the quantiles of the other, typically with a reference line at $y = x$. If the points in the QQ-plot fall entirely along this line, this indicates that the two distributions are the same. In practice, a QQ-plot serves as a measure of similarity between distributions. Generally, what we want to see here is an S-shaped curve. This indicates that the tails of the IC distribution are fatter and contain more information.

QQプロットでは、ICの分布と正規分布の比較ができます。
他の分位との分散を描画。
もしQQプロットの線をずっと下回っていれば、それは２つの分散が一緒という意味。実際にQQプロットは分散を図る方法として使われます。
一般的には、ここではS字カーブを見たいですね。
これは、IC分散が膨らんでいて情報を多く含んでいるという意味になります、



#### IC Heat Plots

The following heat plots show the monthly mean IC, providing us with another visual of consistency.

heat プロットどこ？


<img src="https://3.bp.blogspot.com/-pq-LLjyPoqo/WwJZnND_U-I/AAAAAAABMGc/RPqrox3dUdwSqUo6FWF4zhxzor8kYMIvACLcBGAs/s800/character_turtle_oyako_mago.png" alt="" width="100" height="100">



#### IC by Group

Looking at the returns by quantile for each individual sector in the returns tear sheet helps to show which sectors are driving the bulk of our returns as well as whether quantiles in each sector are broken out how they should be (with the lowest quantiles giving the lowest returns up to the highest quantiles giving the highest returns). If an individual sector has little to no signal (IC), **it makes sense for the quantile returns to be all over the place**. Looking at IC by group helps to make sure that everything is behaving nicely.

セクターごとに収益を見る事は、どのセクターのどのクオンタイルのファクターが収益を伸ばす要因になっているか、どうようにどれが収益を破壊しているか確認できます。

もしあるセクターがシグナルなし（IC）であれば、<img src="https://3.bp.blogspot.com/-pq-LLjyPoqo/WwJZnND_U-I/AAAAAAABMGc/RPqrox3dUdwSqUo6FWF4zhxzor8kYMIvACLcBGAs/s800/character_turtle_oyako_mago.png" alt="" width="100" height="100">



## Turnover

When considering the impact of actually implementing a signal in a strategy, turnover is a critical thing to consider. This plot shows the turnover of the top and bottom quantiles of your factor, the baskets that you would actually be trading on with a long-short approach. Excessive turnover will eat away at the profits of your strategy through commission costs. Sometimes a signal just isn't good enough to fight against the tide on the scale that you have to deal with through your broker.

ストラテジーにシグナルを実装する時、turnoverはクリティカルな要素です。turnover 超過は、コミッションコストという形で収益を食いつぶします。
turnover のコードはこれ。
https://github.com/quantopian/pyfolio/blob/master/pyfolio/txn.py#L149

200％の理由がここにかいてあって、
例えば、SPYからAAPLにすべて資金移動させたとしたら、
SPYを１００％で売る、AAPLを１００％で買うという作業が一日で発生するので
２００％になるという意味になる。



```python 
def get_turnover(positions, transactions, denominator='AGB'):
    """
     - Value of purchases and sales divided
    by either the actual gross book or the portfolio value
    for the time step.
    Parameters
    ----------
    positions : pd.DataFrame
        Contains daily position values including cash.
        - See full explanation in tears.create_full_tear_sheet
    transactions : pd.DataFrame
        Prices and amounts of executed trades. One row per trade.
        - See full explanation in tears.create_full_tear_sheet
    denominator : str, optional
        Either 'AGB' or 'portfolio_value', default AGB.
        - AGB (Actual gross book) is the gross market
        value (GMV) of the specific algo being analyzed.
        Swapping out an entire portfolio of stocks for
        another will yield 200% turnover, not 100%, since
        transactions are being made for both sides.
        - We use average of the previous and the current end-of-period
        AGB to avoid singularities when trading only into or
        out of an entire book in one trading period.
        - portfolio_value is the total value of the algo's
        positions end-of-period, including cash.
    Returns
    -------
    turnover_rate : pd.Series
        timeseries of portfolio turnover rates.
    """
``` 


### Turnover Tear Sheet

We can examine two separate metrics of turnover using Alphalens, percent turnover and factor autocorrelation.

turnover の tear sheet は、turnover 比率と、factor に対する相関を見る

In [ ]:
al.tears.create_turnover_tear_sheet(factor_data);

#### Quantile Turnover

The first table is a plot of turnover for each quantile by period. This indicates the percentage of names that move in and out of each basket.

最初のテーブルは、指定期間ごと、かつファクターのクオンタイルごとにturnover を出します。
これはそれぞれのバスケットの銘柄出し入れにどのくらいTurnoverがかかったか計算しています。

（おそらく、例えば、１DのQuantile1の場合0.080なのは、一日ではQ1の銘柄入れ替えがそんなに起きず、Turnoverのインパクトはそんなに無いことをいっているのかもしれない。
ただ、10日と比べると、0.267ということは、0.080*10で0.8になることと単純に比べる事ができるのかわからない）


#### Factor Autocorrelation

Factor autocorrelation is the measure of correlation between the current value of the factor and its previous value. The idea behind its calculation is to provide another measure of the turnover of the factor quantiles. If the autocorrelation is low, it implies that the current value of the factor has little to do with the previous value and that portfolio positions are **changing frequently from time period to time period**. If the next value of the factor is significantly influenced by its last value, this means that your ranking scheme is more consistent (though this has no influence on its ability to forecast relative price movements).

Factorの自己相関を見る。
以前のFactorと今のFactorの相関関係。
もし、自己相関が低ければ、Time Periodごとにポートフォリオの中がちょくちょく変わることを意味している。

過去と今の相関が高ければ、アルゴリズムのランキングシステムはコンシスタントだということです（価格予測と関係はない）


#### Top and Bottom Quantile Turnover
These plot shows a fairly ordinary turnover for the factor, implying that we will not be hit too hard by the constant changing of portfolio positions, though this varies from time period to time period. We cannot see the costs directly, however, because `Alphalens` does not model commission costs. It simply provides metrics that we can use to judge a factor by itself. To properly model undermining influences such as slippage and commissions you will need to implement a strategy that uses your factor in the backtester.

コンスタントにポートフォリオの中身が変わってしまうと、Turnoverの衝撃が大きい。それを見るためのプロット。



#### Autocorrelation

The second set of plots in the turnover tear sheet is the factor autocorrelation over time. In this case, we have fairly high autocorrelation, corroborating the turnover plots from above that suggest more consistent portfolio positions.

全期間の自己相関。大体高い自己相関。上記の自己相関の補助プロットとしてつかう。


# Full Factor Tear Sheet

We can also generate all of the above plots and statistics in one complete tear sheet. The syntax for generating the full tear sheet all at once is as follows:

In [ ]:
al.tears.create_full_tear_sheet(factor_data, by_group=True);

# More on Factors

Coming up with new factors is all well and good, but often you will need many independently predictive factors to produce a signal that is stable enough to trade. Methods of aggregating factors together will be discussed in future lectures, but the most simple initial approach would just be to normalize the values of each factor you would like to include in the aggregate, add the new normalized factor together, and rank by this new combined factor value.

色々な独立した予測ファクターをシグナルを出すために持ちたい。



# Next Steps

Once you have a factor that looks good, the next step is to implement it in an algorithm. Unfortunately, it isn't enough to simply have a good signal. Trading algorithms have to take into account many other considerations that are not included in `Alphalens`. We need to include how the market at large will react to the trades we're making (market impact/slippage) as well as the transaction costs associated with making those trades. These influences can erode our profits if we do not properly assess their impact through extensive testing.

いいファクターを見つけたら、今度はアルゴリズムに投入だが、アルファレンズでいいファクターを見つけても、それだけでは足りない。
マーケットインパクトやトランザクションコストなど考える事はたくさんある。


To this end, we have the full backtesting environment. It allows for slippage and transaction cost modeling and lets you set limitations for the amount of leverage (debt) that your algorithm can take on to make its trades. Learn more about leverage in [this lecture](https://www.quantopian.com/lectures#Leverage).

We have an [example long-short algorithm](https://www.quantopian.com/lectures#Example:-Long-Short-Equity-Algorithm) that you can clone and use to test your own factors. Try adding in the momentum factor that we played within this lecture to see how the addition of slippage and transaction costs affect the trades made and the resultant profits.

*This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Quantopian, Inc. ("Quantopian"). Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company.  In preparing the information contained herein, Quantopian, Inc. has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Quantopian, Inc. at the time of publication. Quantopian makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.*